In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import seaborn as sns
import os
import sys
plt.rcParams['font.family'] = 'Malgun Gothic'

warnings.filterwarnings('ignore')

In [83]:
sys.path.append("C:/Users/user/github/Function/python/")

In [97]:
from Adaboost_regressor import *
from AdaBoostModel import *

In [15]:
df = pd.read_csv('C:/Users/user/github/Data/capstone/한국서부발전(주)_태양광 발전 현황_20220731.csv', encoding='cp949')
weather = pd.read_csv('C:/Users/user/github/Data/capstone/SURFACE_ASOS_239_HR_2020_2020_2021.csv', encoding='cp949')

In [5]:
df.head(1)

,발전기명,년월일,설비용량(MW),01시,02시,03시,04시,05시,06시,07시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,세종시폐기물매립장태양광,2017-01-01,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,742080.0,334440.0,87240.0,2880.0,4320.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_se = df[df['발전기명']=='세종시폐기물매립장태양광']

In [112]:
df_se['년월일'] = pd.to_datetime(df_se['년월일'])

In [192]:
df_se.tail(1)

,년월일,01시,02시,03시,04시,05시,06시,07시,08시,09시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
2032,2022-07-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.2,42.84,...,47.28,138.24,963.84,237.24,0.0,0.0,0.0,0.0,0.0,0.0


In [219]:
q = df_se.iloc[:,1:]

In [223]:
pd.date_range('2020-01-01','2022-07-31',freq='d')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2022-07-22', '2022-07-23', '2022-07-24', '2022-07-25',
               '2022-07-26', '2022-07-27', '2022-07-28', '2022-07-29',
               '2022-07-30', '2022-07-31'],
              dtype='datetime64[ns]', length=943, freq='D')

In [249]:
tq = df_se[df_se['년월일']>='2020-01-01']

In [250]:
tq.reset_index(inplace=True)

In [251]:
tq.drop(['index'],axis=1, inplace=True)

In [252]:
tqq = tq.iloc[:,1:]

In [253]:
tt = np.ravel(tqq)

In [246]:
len(tt)

22632

In [164]:
from datetime import datetime

In [255]:
time_ = pd.date_range('2020-01-01 01:00:00','2022-08-01 00:00:00',freq='h')

In [258]:
test = pd.DataFrame(time_,columns=['time'])

In [259]:
test['time'] = pd.to_datetime(test['time'])

In [260]:
test['value'] = tt.T

In [266]:
test

,time,value
0,2020-01-01 01:00:00,0.0
1,2020-01-01 02:00:00,0.0
2,2020-01-01 03:00:00,0.0
3,2020-01-01 04:00:00,0.0
4,2020-01-01 05:00:00,0.0
...,...,...
22627,2022-07-31 20:00:00,0.0
22628,2022-07-31 21:00:00,0.0
22629,2022-07-31 22:00:00,0.0
22630,2022-07-31 23:00:00,0.0


In [272]:
tr = test.iloc[:15842]
ts = test.iloc[15842:]

In [352]:
tr_tr = tr.iloc[:,1]

In [354]:
tr_tr = tr_tr.fillna(0)

In [284]:
weather['일시'] = pd.to_datetime(weather['일시'])

In [285]:
we = weather[['일시','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','일조(hr)','일사(MJ/m2)','지면온도(°C)']]

In [286]:
we = we.fillna(0)

In [298]:
total_weather = we[(we['일시']>'2020-01-01') & (we['일시'] <='2022-08-01 00:00:00')]

In [299]:
weather_tr = total_weather.iloc[:15842]
weather_ts = total_weather.iloc[15842:]

In [331]:
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
AdaBoostRegressor.base_estimator_

In [365]:
model = AdaBoostRegressor(n_estimators=1000,random_state=0, learning_rate=0.1)

In [366]:
model.fit(weather_tr.iloc[:,1:], tr_tr)

AdaBoostRegressor(learning_rate=0.1, n_estimators=1000, random_state=0)

In [367]:
model.predict(weather_ts.iloc[:,1:])

array([45794.01718785, 45794.01718785, 45794.01718785, ...,
       52387.02117777, 52472.52731312, 52387.02117777])

In [368]:
model.score(weather_ts.iloc[:,1:], ts.iloc[:,1:])

-1278945.0295936405